In [1]:
import numpy as np
import pandas as pd
import os, random, time, datetime, json
from pandas.io.json import json_normalize
from ast import literal_eval

# Common Model Algorithms
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier

# Common Model Helpers
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics

from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings('ignore')

print(os.listdir("."))

['.DS_Store', 'test_v2.csv', 'sample_submission_v2.csv', 'train_v2.csv', 'train_v2.csv.zip', 'test_v2.csv.zip', '.ipynb_checkpoints', 'ga-customer-revenue-prediction-v1.ipynb', 'sample_submission_v2.csv.zip']


## First, let's explore a small subsample of the data

In [2]:
%%time
small_sample = pd.read_csv('train_v2.csv', nrows=10000)

CPU times: user 1.21 s, sys: 157 ms, total: 1.37 s
Wall time: 1.41 s


In [3]:
print(small_sample.info())
small_sample.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
channelGrouping         10000 non-null object
customDimensions        10000 non-null object
date                    10000 non-null int64
device                  10000 non-null object
fullVisitorId           10000 non-null uint64
geoNetwork              10000 non-null object
hits                    10000 non-null object
socialEngagementType    10000 non-null object
totals                  10000 non-null object
trafficSource           10000 non-null object
visitId                 10000 non-null int64
visitNumber             10000 non-null int64
visitStartTime          10000 non-null int64
dtypes: int64(4), object(8), uint64(1)
memory usage: 1015.7+ KB
None


,channelGrouping,customDimensions,date,device,fullVisitorId,geoNetwork,hits,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,"{""browser"": ""Firefox"", ""browserVersion"": ""not ...",3162355547410993243,"{""continent"": ""Europe"", ""subContinent"": ""Weste...","[{'hitNumber': '1', 'time': '0', 'hour': '17',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508198450,1,1508198450
1,Referral,"[{'index': '4', 'value': 'North America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",8934116514970143966,"{""continent"": ""Americas"", ""subContinent"": ""Nor...","[{'hitNumber': '1', 'time': '0', 'hour': '10',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""referralPath"": ""/a/google.com/transportation...",1508176307,6,1508176307
2,Direct,"[{'index': '4', 'value': 'North America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",7992466427990357681,"{""continent"": ""Americas"", ""subContinent"": ""Nor...","[{'hitNumber': '1', 'time': '0', 'hour': '17',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""(direct)""...",1508201613,1,1508201613
3,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",9075655783635761930,"{""continent"": ""Asia"", ""subContinent"": ""Western...","[{'hitNumber': '1', 'time': '0', 'hour': '9', ...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508169851,1,1508169851
4,Organic Search,"[{'index': '4', 'value': 'Central America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",6960673291025684308,"{""continent"": ""Americas"", ""subContinent"": ""Cen...","[{'hitNumber': '1', 'time': '0', 'hour': '14',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508190552,1,1508190552


As we can see, there are 4 columns with JSON data:
- device
- geoNetwork
- hits
- totals
- trafficSource

Let's write a function to load this data and flatten these JSON data in columns. The work will be based on [Leonardo Ferreira's kernel](https://www.kaggle.com/kabure/exploring-the-consumer-patterns-ml-pipeline).

## Flattening JSON columns and loading a random subset of the data

In [4]:
# json_columns: Columns that have json format
# p is a fractional number to skiprows and read just a random sample of the our dataset. 
def json_read(filepath, p=0.07, parseHitsAndCustomDimensions=False):
    json_columns = ['device', 'geoNetwork', 'totals', 'trafficSource']
    df = pd.read_csv(filepath, 
                     converters={column: json.loads for column in json_columns}, # loading the json columns properly
                     dtype={'fullVisitorId': 'str'}, # transforming this column to string
                     skiprows=lambda i: i>0 and random.random() > p)# Number of rows that will be imported randomly
    
    if parseHitsAndCustomDimensions:
        # Parse hits column
        df['hits'] = df['hits'].apply(literal_eval)
        df['hits'] = df['hits'].str[0]
        to_replace_keys = ['time',
         'hour',
         'minute',
         'isInteraction',
         'isEntrance',
         'isExit',
         'referer',
         'page',
         'transaction',
         'item',
         'appInfo',
         'exceptionInfo',
         'product',
         'promotion',
         'eCommerceAction',
         'experiment',
         'customVariables',
         'customDimensions',
         'customMetrics',
         'type',
         'social',
         'contentGroup',
         'dataSource',
         'publisher_infos']

        to_replace = {key:np.NaN for key in to_replace_keys}
        df['hits'] = df['hits'].apply(lambda x: to_replace if pd.isnull(x) else x)

        # Parse customDimensions column
        df['customDimensions'] = df['customDimensions'].apply(literal_eval)
        df['customDimensions'] = df['customDimensions'].str[0]
        df['customDimensions'] = df['customDimensions'].apply(lambda x: {'index':np.NaN,'value':np.NaN} if pd.isnull(x) else x)

        json_columns = ['device', 'geoNetwork', 'totals', 'trafficSource','hits','customDimensions']
    
    
    for column in json_columns: #loop to finally transform the columns in data frame
        #It will normalize and set the json to a table
        column_as_df = json_normalize(df[column]) 
        # here will be set the name using the category and subcategory of json columns
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns] 
        # after extracting the values, let drop the original columns
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
        
    print(f"Loaded {os.path.basename(filepath)}. Shape: {df.shape}")
    return df

In [5]:
%%time
train = json_read('train_v2.csv', p=0.03)

Loaded train_v2.csv. Shape: (51699, 59)
CPU times: user 44.7 s, sys: 7.98 s, total: 52.7 s
Wall time: 57.5 s


In [6]:
print(train.info())
pd.set_option('display.max_columns', None) # To show all columns in .head()
train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51699 entries, 0 to 51698
Data columns (total 59 columns):
channelGrouping                                      51699 non-null object
customDimensions                                     51699 non-null object
date                                                 51699 non-null int64
fullVisitorId                                        51699 non-null object
hits                                                 51699 non-null object
socialEngagementType                                 51699 non-null object
visitId                                              51699 non-null int64
visitNumber                                          51699 non-null int64
visitStartTime                                       51699 non-null int64
device.browser                                       51699 non-null object
device.browserSize                                   51699 non-null object
device.browserVersion                                51699 non-null ob

,channelGrouping,customDimensions,date,fullVisitorId,hits,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.browserSize,device.browserVersion,device.deviceCategory,device.flashVersion,device.isMobile,device.language,device.mobileDeviceBranding,device.mobileDeviceInfo,device.mobileDeviceMarketingName,device.mobileDeviceModel,device.mobileInputSelector,device.operatingSystem,device.operatingSystemVersion,device.screenColors,device.screenResolution,geoNetwork.city,geoNetwork.cityId,geoNetwork.continent,geoNetwork.country,geoNetwork.latitude,geoNetwork.longitude,geoNetwork.metro,geoNetwork.networkDomain,geoNetwork.networkLocation,geoNetwork.region,geoNetwork.subContinent,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.sessionQualityDim,totals.timeOnSite,totals.totalTransactionRevenue,totals.transactionRevenue,totals.transactions,totals.visits,trafficSource.adContent,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.criteriaParameters,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,Organic Search,[],20171016,6723102816157399269,"[{'hitNumber': '1', 'time': '0', 'hour': '12',...",Not Socially Engaged,1508180825,1,1508180825,Chrome,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Europe,Romania,not available in demo dataset,not available in demo dataset,not available in demo dataset,rdsnet.ro,not available in demo dataset,not available in demo dataset,Eastern Europe,NaN,3,1,3,1,24,NaN,NaN,NaN,1,NaN,NaN,not available in demo dataset,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
1,Organic Search,"[{'index': '4', 'value': 'APAC'}]",20171016,5855400646126579899,"[{'hitNumber': '1', 'time': '0', 'hour': '9', ...",Not Socially Engaged,1508173068,1,1508173068,Safari,not available in demo dataset,not available in demo dataset,mobile,not available in demo dataset,True,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,iOS,not available in demo dataset,not available in demo dataset,not available in demo dataset,Singapore,not available in demo dataset,Asia,Singapore,not available in demo dataset,not available in demo dataset,(not set),singnet.com.sg,not available in demo dataset,(not set),Southeast Asia,NaN,4,1,4,1,44,NaN,NaN,NaN,1,NaN,NaN,not available in demo dataset,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
2,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,520531418264435064,"[{'hitNumber': '1', 'time': '0', 'hour': '12',...",Not Socially Engaged,1508183418,1,1508183418,Chrome,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Macintosh,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Europe,Netherlands,not available in demo dataset,not available in demo dataset,not available in demo dataset,ziggo.nl,not available in demo dataset,not available in demo dataset,Western Europe,NaN,4,1,3,1,34,NaN,NaN,NaN,1,NaN,NaN,not available in demo datase

## Checking missing values

In [18]:
def missing_values(data):
    total = data.isnull().sum().sort_values(ascending = False) # getting the sum of null values and ordering
    percent = (data.isnull().sum() / data.isnull().count() * 100 ).sort_values(ascending = False) #getting the percent and order of null
    df = pd.concat([total, percent], axis=1, keys=['Total', 'Percent']) # Concatenating the total and percent
    df = df[df.Total > 0]
    return df

In [19]:
pd.set_option('display.max_rows', None) # To show all columns in .head()
missing_values(train)

,Total,Percent
totals.totalTransactionRevenue,51140,98.918741
totals.transactions,51138,98.914873
trafficSource.adContent,49682,96.098571
trafficSource.adwordsClickInfo.page,49360,95.475735
trafficSource.adwordsClickInfo.slot,49360,95.475735
trafficSource.adwordsClickInfo.adNetworkType,49360,95.475735
trafficSource.adwordsClickInfo.gclId,49357,95.469932
trafficSource.referralPath,34603,66.931662
trafficSource.keyword,31861,61.627884
totals.timeOnSite,26396,51.057080


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 95 entries, hits.exceptionInfo to totals.pageviews
Data columns (total 2 columns):
Total      95 non-null int64
Percent    95 non-null float64
dtypes: float64(1), int64(1)
memory usage: 2.2+ KB


## Filling missing values

In [16]:
def fill_na_values(df):
    df['totals.pageviews'].fillna(1, inplace=True)
    df['totals.newVisits'].fillna(0, inplace=True)
    df['totals.bounces'].fillna(0, inplace=True)
    df["totals.transactionRevenue"] = df["totals.transactionRevenue"].fillna(0.0).astype(float) #filling NA with zero
    df['trafficSource.isTrueDirect'].fillna(False, inplace=True) # filling boolean with False
    df['trafficSource.adwordsClickInfo.isVideoAd'].fillna(True, inplace=True) # filling boolean with True
    df['totals.pageviews'] = df['totals.pageviews'].astype(int) # setting numerical column as integer
    df['totals.newVisits'] = df['totals.newVisits'].astype(int) # setting numerical column as integer
    df['totals.bounces'] = df['totals.bounces'].astype(int)  # setting numerical column as integer
    df["totals.hits"] = df["totals.hits"].astype(float) # setting numerical to float
    df['totals.visits'] = df['totals.visits'].astype(int) # seting as int

    return df #return the transformed dataframe

In [17]:
train = fill_na_values(train)

In [30]:
df = missing_values(train)
df.index.values

array(['totals.totalTransactionRevenue', 'totals.transactions',
       'trafficSource.adContent', 'trafficSource.adwordsClickInfo.page',
       'trafficSource.adwordsClickInfo.slot',
       'trafficSource.adwordsClickInfo.adNetworkType',
       'trafficSource.adwordsClickInfo.gclId',
       'trafficSource.referralPath', 'trafficSource.keyword',
       'totals.timeOnSite', 'totals.sessionQualityDim'], dtype=object)

## Feature engineering: dates

In [22]:
from datetime import datetime

def date_process(df):
    df["date"] = pd.to_datetime(df["date"], format="%Y%m%d")
    df["_weekday"] = df['date'].dt.weekday
    df["_day"] = df['date'].dt.day
    df["_month"] = df['date'].dt.month
    df["_year"] = df['date'].dt.year
    df['_visitHour'] = (df['visitStartTime'].apply(lambda x: str(datetime.fromtimestamp(x).hour))).astype(int)
    return df

In [23]:
train = date_process(train)

In [31]:
train[df.index.values].head()

,totals.totalTransactionRevenue,totals.transactions,trafficSource.adContent,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.gclId,trafficSource.referralPath,trafficSource.keyword,totals.timeOnSite,totals.sessionQualityDim
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(not provided),24,1
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(not provided),44,1
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(not provided),34,1
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/,NaN,34,1
4,NaN,NaN,NaN,1,Top,Google Search,EAIaIQobChMIvIzv1OP21gIVUbXACh2KBg8TEAAYASAAEg...,NaN,1X4Me6ZKNV0zg-jV,359,1
